![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_CamemBertForZeroShotClassification.ipynb)

## Import ONNX CamemBertForZeroShotClassification models from HuggingFace 🤗  into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `CamemBertForZeroShotClassification` is only available since in `Spark NLP 5.4.2` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import CamemBERT models trained/fine-tuned for sequence classification via `CamembertForSequenceClassification` or `TFCamembertForSequenceClassification`. These models are usually under `Sequence Classification` category and have `camembert` in their labels
- Reference: [TFCamembertForSequenceClassification](https://huggingface.co/docs/transformers/model_doc/camembert#transformers.TFCamembertForSequenceClassification)
- Some [example models](https://huggingface.co/models?other=camembert&pipeline_tag=token-classification)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.51.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- CamembertTokenizer requires the `SentencePiece` library, so we install that as well

In [ ]:
!pip install -q transformers[onnx]==4.51.3 optimum onnx

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [tblard/tf-allocine](https://huggingface.co/tblard/tf-allocine)  model from HuggingFace as an example and load it as a `ORTModelForSequenceClassification`, representing an ONNX model.
- In addition to the CamemBERT model, we also need to save the `CamembertTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [3]:
from transformers import CamembertTokenizer
from optimum.onnxruntime import ORTModelForSequenceClassification

MODEL_NAME = 'mtheo/camembert-base-xnli'
ONNX_MODEL = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForSequenceClassification.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(ONNX_MODEL)

tokenizer = CamembertTokenizer.from_pretrained('camembert-base')
tokenizer.save_pretrained(ONNX_MODEL)

('onnx_models/mtheo/camembert-base-xnli/tokenizer_config.json',
 'onnx_models/mtheo/camembert-base-xnli/special_tokens_map.json',
 'onnx_models/mtheo/camembert-base-xnli/sentencepiece.bpe.model',
 'onnx_models/mtheo/camembert-base-xnli/added_tokens.json')

Let's have a look inside these two directories and see what we are dealing with:

In [5]:
!ls -l {ONNX_MODEL}

total 433252
-rw-r--r-- 1 root root        22 Jun 11 21:49 added_tokens.json
-rw-r--r-- 1 root root       885 Jun 11 21:49 config.json
-rw-r--r-- 1 root root 442816472 Jun 11 21:49 model.onnx
-rw-r--r-- 1 root root    810912 Jun 11 21:49 sentencepiece.bpe.model
-rw-r--r-- 1 root root       374 Jun 11 21:49 special_tokens_map.json
-rw-r--r-- 1 root root      1814 Jun 11 21:49 tokenizer_config.json


- As you can see, we need to move `sentencepiece.bpe.model` from the tokenizer to assets folder which Spark NLP will look for
- In addition to vocabs, we also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [6]:
!mkdir -p {ONNX_MODEL}/assets

labels = ort_model.config.id2label
sorted_labels = [label for _, label in sorted(labels.items())]

with open(f"{ONNX_MODEL}/assets/labels.txt", 'w') as f:
    f.write('\n'.join(sorted_labels))

!mv {ONNX_MODEL}/sentencepiece.bpe.model {ONNX_MODEL}/assets/

In [8]:
!ls -lR {ONNX_MODEL}

onnx_models/mtheo/camembert-base-xnli:
total 432464
-rw-r--r-- 1 root root        22 Jun 11 21:49 added_tokens.json
drwxr-xr-x 2 root root      4096 Jun 11 21:50 assets
-rw-r--r-- 1 root root       885 Jun 11 21:49 config.json
-rw-r--r-- 1 root root 442816472 Jun 11 21:49 model.onnx
-rw-r--r-- 1 root root       374 Jun 11 21:49 special_tokens_map.json
-rw-r--r-- 1 root root      1814 Jun 11 21:49 tokenizer_config.json

onnx_models/mtheo/camembert-base-xnli/assets:
total 796
-rw-r--r-- 1 root root     32 Jun 11 21:50 labels.txt
-rw-r--r-- 1 root root 810912 Jun 11 21:49 sentencepiece.bpe.model


Voila! We have our `sentencepiece.bpe.model` and `labels.txt` inside assets directory

## Import and Save CamemBertForZeroShotClassification in Spark NLP


- Let's install and setup Spark NLP in Google Colab.
- For this example we'll use specific versions of `pyspark` and `spark-nlp` that we've already tested with this transformer model to make sure everything runs smoothly.

In [9]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 31.8 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [10]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `CamemBertForSequenceClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `CamemBertForSequenceClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [11]:
from sparknlp.annotator import CamemBertForZeroShotClassification

zero_shot_classifier = CamemBertForZeroShotClassification.loadSavedModel(
      f"{ONNX_MODEL}",
      spark
  )\
    .setInputCols(["document",'token'])\
    .setOutputCol("class")\
    .setCaseSensitive(True)\
    .setMaxSentenceLength(128)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [12]:
zero_shot_classifier.write().overwrite().save("./{}_spark_nlp_onnx".format(ONNX_MODEL))

Let's clean up stuff we don't need anymore

In [13]:
!rm -rf {ONNX_MODEL}

Awesome 😎  !

This is your CamemBertForZeroShotClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [14]:
! ls -l {ONNX_MODEL}_spark_nlp_onnx

total 433312
-rw-r--r-- 1 root root 442884176 Jun 11 21:52 camembert_classification_onnx
-rw-r--r-- 1 root root    810912 Jun 11 21:52 camembert_spp
drwxr-xr-x 3 root root      4096 Jun 11 21:52 fields
drwxr-xr-x 2 root root      4096 Jun 11 21:52 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny CamemBertForZeroShotClassification model 😊

In [15]:
zero_shot_classifier_loaded = CamemBertForZeroShotClassification.load(
    f"./{ONNX_MODEL}_spark_nlp_onnx"
) \
.setInputCols(["document", "token"]) \
.setOutputCol("class") \
.setCandidateLabels(["sport", "politique", "science"])

You can see what labels were used to train this model via `getClasses` function:

In [19]:
zero_shot_classifier_loaded.getClasses()

['entailment', 'neutral', 'contradiction']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [17]:
from pyspark.ml import Pipeline
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    zero_shot_classifier_loaded
])

example = spark.createDataFrame([
    ["Alad'2 est clairement le meilleur film de l'année 2018."],
    ["Je m'attendais à mieux de la part de Franck Dubosc !"]
], ["text"])

result = pipeline.fit(example).transform(example)
result.select("text", "class.result").show(truncate=False)

+-------------------------------------------------------+-----------+
|text                                                   |result     |
+-------------------------------------------------------+-----------+
|Alad'2 est clairement le meilleur film de l'année 2018.|[politique]|
|Je m'attendais à mieux de la part de Franck Dubosc !   |[politique]|
+-------------------------------------------------------+-----------+



That's it! You can now go wild and use hundreds of `CamemBertForZeroShotClassification` models from HuggingFace 🤗 in Spark NLP 🚀
